# Gemma3



Original [Infotrend's CoreAI](https://github.com/Infotrend-Inc/CoreAI) Demo project: https://github.com/Infotrend-Inc/CoreAI-DemoProjects/tree/main/Gemma3

## Description

This notebook uses Google’s Gemma 3 open models locally (using its instruction-tuned checkpoint: `gemma-3-4b-it`) to show Large Language Model (LLM) and Vision Language Model (VLM) capabilities. without using a service such as Ollama to `serve` the model.

High-level steps:

1. Install/load runtime packages.  ￼
2. Initialize tokenizer and model
3. Prompting (text): a small chat message (system/user) to answer a question.
4. Multimodal (image): perform image understanding on a provided image.

## Pre-requisites

To support features of this notebook with CoreAI, we need to install some libraries that are not pre-installed but are required. 

**Create and Activate the Virtual Environment:**

Open a terminal within the Jupyter notebook (`File -> New -> Terminal`).
Navigate to this project's folder; where we want to set up the environment (where this notebook is located) and run:


```bash
export PROJECT_NAME="Gemma3"
export PIP_CACHE_DIR=`pwd`/.cache/pip
mkdir -p $PIP_CACHE_DIR
python -m venv --system-site-packages myvenv
source myvenv/bin/activate
pip install ipykernel
python -m ipykernel install --user --name=${PROJECT_NAME}-myvenv --display-name="Python (${PROJECT_NAME}-myvenv)"
echo ""; echo "Before continuing load the created Python kernel: Python (${PROJECT_NAME}-myvenv)"
```

This will create a local virtual environment to contain installed files to the mounted `/iti` folder (and not modify the container's files).

**Load the Python kernel described above before running the cell below** (it might take a few seconds for the kernel to appear in the list of kernels).

Install the required Libraries (from `requirements.txt`).
The rest of this notebook relies on the proper kernel to be loaded and environment variables to be set. 

In [ ]:
!. ./myvenv/bin/activate && pip install -r requirements.txt

Add `accelerate` to the PATH

In [ ]:
import os
pwd = os.getcwd()
os.environ['PATH'] =  os.path.join(pwd, 'myvenv/bin') + os.pathsep + os.environ['PATH']

! echo $PATH
! which accelerate

**Make sure `HF_HOME` is set BEFORE `transformers` is loaded**

In [ ]:
import os
os.makedirs('HF_HOME', exist_ok=True)
os.environ['HF_HOME'] = 'HF_HOME'

By default, the notebook will hide dot files (i.e. `.env`).
Edit the `env.example` file included in this notebook to reflect the required API keys and rename the file `.env` from a "Terminal".

In [ ]:
# Load environment configuration file
# This sets up the basic structure for API credentials

import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if 'HF_TOKEN' not in os.environ:
    printf("No HF_TOKEN set, will not be able to download the model")
    exit(1)

hf_token=os.environ['HF_TOKEN']

# LLM: Large Language Model queries

## LLM: Obtain model

In [ ]:
from transformers import pipeline
import torch

pipe = pipeline(
    "text-generation",
    model="google/gemma-3-4b-it",
    device="cuda",
    torch_dtype=torch.bfloat16,
    token=hf_token
)


## LLM: Question Answering

In [ ]:
question = "What is OpenStack? What is its landscape? What is the scientific SIG?"

messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "you are a helpful assistant"}]
    },
    {
        "role": "user",
        "content": [{"type": "text", "text": f"{question}"}]
    },
]
user_message = messages[1]["content"][0]["text"]

output = pipe(text_inputs=user_message, max_new_tokens=4000)
print(output[0]["generated_text"])

### When done using the LLM: free up GPU memory (when running on 24GB VRAM)

Note: we are using the same variable for text and image (`pipe`) this code can be used when switching from one method to the next

In [ ]:
# Free up the model from memory before testing the VLM (Garbage Collection)
import gc

# check memory
print(torch.cuda.memory_allocated())

del pipe

gc.collect()
torch.cuda.empty_cache()

# check memory again
print(torch.cuda.memory_allocated())

# VLM: Image Understanding

## VLM: Obtain model

In [ ]:
from transformers import pipeline
import torch

pipe = pipeline(
    "image-text-to-text",
    model="google/gemma-3-4b-it",
    device="cuda",
    torch_dtype=torch.bfloat16,
    token=hf_token
)

## VLM: Exract information from images

Adapt `img_file` to the file to be analyzed.

Modify the prompt to answer different types of questions.

In [ ]:
import PIL 
import base64
import io
from IPython.display import display

img_file = "data/openstack-map-v20250401.png"


def base64_image(img_file):
    img_type = "png"
    img_b64 = None
    img_str = None
    img_bytes = io.BytesIO()
    with PIL.Image.open(img_file) as image:
        display(image)
        image.save(img_bytes, format=img_type)
        img_b64 = base64.b64encode(img_bytes.getvalue()).decode('utf-8')
    if img_b64 is not None:
        img_str = f"data:image/{img_type};base64,{img_b64}"
    
    if img_str is None:
        print("No valid image data")
        exit(1)

    return img_str

img_str = base64_image(img_file)

messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are a helpful assistant."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image_url", "image_url": { "url": img_str } },
            {"type": "text", "text": "Describe the image. what is the field of expertise needed, explain the idea behind the meaning of the image?"}
        ]
    }
]

output = pipe(text=messages, max_new_tokens=1000)
print(output[0]["generated_text"][-1]["content"])